In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
from scipy.integrate import solve_ivp
from scipy import optimize
from scipy import linalg
import scipy.optimize as so

In [2]:
# Unless specified otherwise, s will refer to the array representing the susceptible populations. Similarly, for i and r.

Let us say the original transmission matrix is 
$\begin{bmatrix}
b_{11} & b_{12} & b_{13}\\
b_{21} & b_{22} & b_{23}\\
b_{31} & b_{32} & b_{33}
\end{bmatrix}$
$b_{11}$ is the rate at which an individual in group 1 infects another individual in group 1
$b_{12}$ is the rate at which an individual in group 2 infects another individual in group 1 and so on....

An intervention of type 1, leads to a new matrix: 
$\begin{bmatrix}
b_{11}(1-c_1) & b_{12}(1-c_1) & b_{13}(1-c_1)\\ 
b_{21}(1-c_2) & b_{22}(1-c_2) & b_{23}(1-c_2)\\
b_{31}(1-c_3) & b_{32}(1-c_3) & b_{33}(1-c_3)
\end{bmatrix}$
An intervention of type 2, leads to a new matrix: 
$\begin{bmatrix}
b_{11}(1-c_1)^2 & b_{12}(1-c_1)(1-c_2) & b_{13}(1-c_1)(1-c_3)\\ 
b_{21}(1-c_2)(1-c_1) & b_{22}(1-c_2)^2 & b_{23}(1-c_2)(1-c_3)\\
b_{31}(1-c_3)(1-c_1) & b_{32}(1-c_3)(1-c_2) & b_{33}(1-c_3)^2
\end{bmatrix}$

Now, the NGM is just the above matrix elements multiplied by their respective gorup sizes and divided by $\gamma$ (recovery rate).

The final size relations make use of this NGM. So, we choose our required final sizes, and solve for $c_1, c_2, c_3$

In [3]:
#Choose intervention type -- 1 or 2
#Type 1: Intervention creates a protective effect for susceptible. b_ij -> b_ij (1-c_i)
#Type 2: Intervention creates both a protective effect for susceptible and a preventive effect for infectious. b_ij -> b_ij (1-c_i)(1-c_j)

type = 2

#Choose if intervention should only reduce transmission. If false, the transmission may be increased i.e. c can be negative
transmission_reduce = True

In [4]:
def growth_rate(s, b, gamma): #This function returns the growth rate
    B = (b.T * s).T 
    M = B - np.eye(len(s))*gamma #The largest eigenvalue of M is the growth rate of the epidemic
    EV = linalg.eigvals(M)
    gr = np.amax(EV) #Growth rate
    return gr 

In [5]:
def FS(x, b, n, g): ## This function gives the final size using the above defined parameters
    return [x[k] - n[k]*(1-np.exp(-np.sum(b[k]*x/g))) for k in range(len(n))]

In [6]:
def FS_two_waves(x, b, r, n, g):
    return [(n[k]-x[k])/(n[k] - r[k]) - np.exp(-np.sum(b[k]*(x-r))/g) for k in range(len(n))]

In [7]:
def LCFS(a, x): #Expression for linear combination of final sizes 'x'.
    return np.sum(a*x) #a and x are arrays with three elements

In [8]:
def c_typeI(r, b, n):
    #c = np.array([1 + np.log(1-r[0]/n[0])/np.sum(b[0]*r), 1 + np.log(1-r[1]/n[1])/np.sum(b[1]*r), 1 + np.log(1-r[2]/n[2])/np.sum(b[2]*r)])
    c = [1 + np.log(1-r[k]/n[k])/np.sum(b[k]*r) for k in range(len(n))]
    return np.array(c)
    
def c_typeII_eqn(c, b, r, n):
    #return np.array([np.sum(b[0]*(1-c[0])*(1-c)*r) + np.log(1-r[0]/n[0]), np.sum(b[1]*(1-c[1])*(1-c)*r) + np.log(1-r[1]/n[1]), np.sum(b[2]*(1-c[2])*(1-c)*r) + np.log(1-r[2]/n[2])])
    return np.array([np.sum(b[k]*(1-c[k])*(1-c)*r) + np.log(1-r[k]/n[k]) for k in range(len(n))])
    
def c_typeII(r, b, n):
    sol = so.root(c_typeII_eqn, 0.8*np.ones(len(n)), args = (b, r, n)) #Using a numerical solver to find the c values
    return (sol.x, sol.success)
    

In [9]:
##Showing all possible strategies for a given matrix

In [10]:
#n = np.array([0.5, 0.5])
n = np.array([0.7, 0.3])
no_g = len(n)
gamma = 1 #Recovery rate
A = np.array([1, 1])
#b = np.random.uniform(low = 0, high = 4, size = (no_g, no_g)) #Transmission parameters i.e. beta_ij for groups i and j
#b = np.array([[4, 2], [1, 3]])
b = np.array([[2, 3.5], [2.5, 1.5]])
print('NGM = ', (b.T * n)/gamma)

constraint_growth_rate = optimize.NonlinearConstraint(lambda x: growth_rate(n-x, b, gamma), lb=0, ub=0) #since x is final size, we pass the s array = n-x to the growth rate function
constraint_universal = optimize.NonlinearConstraint(lambda x: c_typeII(x, b, n)[0], ub=0.95, lb = -np.inf)
constraint_reduce_transmission = optimize.NonlinearConstraint(lambda x: c_typeII(x, b, n)[0], ub=0.95, lb=0) #Edited the upper bound to 0.95. Transmission can only be reduced by 95 % at max. 100 % is not allowed because it either involves extremely high costs or is not feasible

root_ord = so.root(FS, 0.9*n, args = (b, n, gamma)) #Using a numerical solver to find the final size
FS_ord = root_ord.x #The final sizes without intervention

NGM =  [[1.4  0.75]
 [2.45 0.45]]


In [11]:
#Finding the unconstrained solution
count_guess = 0
max_guesses = 40
possible_sizes = []
possible_red_facs = []
for count_guess in range(max_guesses):
    optimize_ic = np.random.rand(no_g)*n
    min_size_unconstrained = optimize.minimize(LCFS, x0 = optimize_ic, args = (A), constraints = (constraint_growth_rate, constraint_universal), bounds = [(0, n[k]) for k in range(len(n))])
    if min_size_unconstrained.success == True:
        c, soln_exists = c_typeII(min_size_unconstrained.x, b, n)
        if soln_exists == True:
            possible_sizes.append(min_size_unconstrained.x)
            possible_red_facs.append(c)
if possible_sizes == []:
    print('Can not find an unconstrained final size')
else:
    FS_unconstrained = possible_sizes[np.argmin(np.sum(possible_sizes, 1))]
    c_unconstrained = possible_red_facs[np.argmin(np.sum(possible_sizes, 1))]
    b_unconstrained = np.array([b[k]*(1-c_unconstrained[k])*(1-c_unconstrained) for k in range(len(n))])  ## The new transmission matrix
    if np.any(c_unconstrained>0.95): 
        print('b = ', b_unconstrained)
        print('Could not find valid reduction factors: c = ', c_unconstrained)



C:\Users\Pratyush\Miniconda3\envs\LTURA\lib\site-packages\scipy\optimize\_numdiff.py:579: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
C:\Users\Pratyush\Miniconda3\envs\LTURA\lib\site-packages\scipy\optimize\slsqp.py:427: ComplexWarning: Casting complex values to real discards the imaginary part
  slsqp(m, meq, x, xl, xu, fx, c, g, a, acc, majiter, mode, w, jw,
C:\Users\Pratyush\AppData\Local\Temp/ipykernel_19064/3267254621.py:8: RuntimeWarning: divide by zero encountered in log
  return np.array([np.sum(b[k]*(1-c[k])*(1-c)*r) + np.log(1-r[k]/n[k]) for k in range(len(n))])


In [12]:
FS_unconstrained, c_unconstrained

(array([0.20000002, 0.29999999]), array([ 0.94811043, -5.15584358]))

In [13]:
#Finding the constrained solution
count_guess = 0
max_guesses = 40
possible_sizes = []
possible_red_facs = []
for count_guess in range(max_guesses):
    optimize_ic = np.random.rand(no_g)*n
    min_size_constrained = optimize.minimize(LCFS, x0 = optimize_ic, args = (A), constraints = (constraint_growth_rate, constraint_reduce_transmission), bounds = [(0, n[k]) for k in range(len(n))])
    if min_size_constrained.success == True:
        c, soln_exists = c_typeII(min_size_constrained.x, b, n)
        if soln_exists == True:
            possible_sizes.append(min_size_constrained.x)
            possible_red_facs.append(c)
if possible_sizes == []:
    print('Can not find an unconstrained final size')
else:
    FS_constrained = possible_sizes[np.argmin(np.sum(possible_sizes, 1))]
    c_constrained = possible_red_facs[np.argmin(np.sum(possible_sizes, 1))]
    b_constrained = np.array([b[k]*(1-c_constrained[k])*(1-c_constrained) for k in range(len(n))])  ## The new transmission matrix
    if np.any(c_constrained>0.95): 
        print('b = ', b_constrained)
        print('Could not find valid reduction factors: c = ', c_constrained)



In [14]:
print('FS_ord, unconstrained new size, constrained new size = ', FS_ord, FS_unconstrained, FS_constrained)
print('FS_ord, sum unconstrained new size, sum constrained new size = ', np.sum(FS_ord), np.sum(FS_unconstrained), np.sum(FS_constrained))
print('c values: unconstrained and constrained', c_unconstrained, c_constrained)

FS_ord, unconstrained new size, constrained new size =  [0.61688523 0.25631069] [0.20000002 0.29999999] [0.39041556 0.18389686]
FS_ord, sum unconstrained new size, sum constrained new size =  0.8731959213801819 0.500000011928698 0.5743124243958009
c values: unconstrained and constrained [ 0.94811043 -5.15584358] [ 3.10008804e-01 -2.86213972e-13]


In [15]:
%matplotlib widget

In [16]:
if no_g == 2:
    s0, s1 = np.arange(0.00, n[0]+0.005, 0.005), np.arange(0.00, n[1]+0.005, 0.005)
    S0, S1 = [], []
    growth_rate_list, epidemic_size_list = [], []
    [(S0.append(x), S1.append(y), growth_rate_list.append(growth_rate([x, y], b, gamma))) for x in s0 for y in s1]
    [(epidemic_size_list.append(np.sum(so.root(FS_two_waves, 0.9*n, args = (b, n - np.array([x, y]), n, gamma)).x))) for x in s0 for y in s1]
    growth_rate_list = np.array(growth_rate_list)
    S0 = np.array(S0)
    S1 = np.array(S1)
    growth_rate_matrix = np.real(np.reshape(growth_rate_list, (len(s0), len(s1))))
    epidemic_size_matrix = np.real(np.reshape(epidemic_size_list, (len(s0), len(s1))))
    thresh = np.where((growth_rate_list<0.005) & (growth_rate_list>-0.005))[0] #Looks for growth rate = 0 i.e. threshold

C:\Users\Pratyush\AppData\Local\Temp/ipykernel_19064/2624452448.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  return [(n[k]-x[k])/(n[k] - r[k]) - np.exp(-np.sum(b[k]*(x-r))/g) for k in range(len(n))]


In [17]:
if no_g == 2:
    fig, ax = plt.subplots(1, 2, figsize = (10, 5)) 
    #im1 = ax.scatter(growth_rate_list[0], growth_rate_list[1], c = growth_rate_list[2], s = 4, marker = 's', cmap = 'viridis_r')
    im1 = ax[0].imshow(growth_rate_matrix.T, cmap = 'inferno', extent = (s0[0], s0[-1], s1[0], s1[-1]), origin = 'lower', interpolation = 'none', aspect = 'auto')
    im2 = ax[1].imshow(epidemic_size_matrix.T, cmap = 'copper', extent = (s0[0], s0[-1], s1[0], s1[-1]), origin = 'lower', interpolation = 'none', aspect = 'auto')
    
    

    ax[0].plot(S0[thresh], S1[thresh], c = 'k', lw = 0, marker = '.', ls = '-.', markersize = 2) 
    ax[1].plot(S0[thresh], S1[thresh], c = 'red', lw = 1, ls = '-.') 
    #ax[1].scatter(S0[thresh], S1[thresh], c = 'k', marker = ',', s = 0.25) 
        
    c_vis_list = [c_unconstrained, c_constrained]
    marker_shape = ['s', 'd']
    if type == 2:
        for j, c_vis in enumerate(c_vis_list):
            c_vis = np.array(c_vis)
            b_vis = np.array([b[k]*(1-c_vis[k])*(1-c_vis) for k in range(len(n))])  ## The new transmission matrix
            root_vis = so.root(FS, 0.9*n, args = (b_vis, n, gamma))
            s_vis = n - root_vis.x #The final sizes with intervention
            ax[0].scatter(s_vis[0], s_vis[1], label = r'$c = (%.2f$, $%.2f)$'%(c_vis[0], c_vis[1]), facecolor = 'none', edgecolor = 'thistle', marker = marker_shape[j])
            print(c_vis, s_vis)
            print('b = ', b_vis)

    check_list = thresh#[0::5]
    temp = []
    for j, check in enumerate(check_list):
        c_check, c_status = c_typeII([n[0] - S0[check], n[1]-S1[check]], b, n)
        if c_status == True and np.all(c_check>-1e-3) and np.all(c_check<1):
            temp.append(check)
        #else:
        #    print('failed c = ', c_check)
    ax[1].plot(S0[temp], S1[temp], c = 'green', lw = 2, ls = '-') 
    #ax[0].plot(growth_rate_list[0][thresh], growth_rate_list[1][thresh], c = 'k', lw = 0.5, marker = ',', ls = '-.') 
    #ax[1].plot(growth_rate_list[0][thresh], growth_rate_list[1][thresh], c = 'k', lw = 1) 
        
    fig.colorbar(im1, ax=ax[0], location = 'bottom')
    fig.colorbar(im2, ax=ax[1], location = 'bottom')
    ax[0].legend(framealpha = 0.3, fancybox = False)
    ax[0].set_xlabel(r'$s_1^*$')
    ax[0].set_ylabel(r'$s_2^*$')
    ax[1].set_xlabel(r'$s_1^*$')
    ax[1].set_ylabel(r'$s_2^*$')
    ax[1].set_xlim(left = 0.01)
    ax[1].set_ylim(bottom = 0.01)
    fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.94811043 -5.15584358] [4.99999978e-01 1.00452204e-08]
b =  [[5.38505496e-03 1.11798427e+00]
 [7.98560192e-01 5.68416154e+01]]
[ 3.10008804e-01 -2.86213972e-13] [0.30958444 0.11610314]
b =  [[0.9521757  2.41496919]
 [1.72497799 1.5       ]]


C:\Users\Pratyush\AppData\Local\Temp/ipykernel_19064/3267254621.py:8: RuntimeWarning: divide by zero encountered in log
  return np.array([np.sum(b[k]*(1-c[k])*(1-c)*r) + np.log(1-r[k]/n[k]) for k in range(len(n))])


In [29]:
if no_g == 2:
    fig, ax = plt.subplots(1, 2, figsize = (10, 5)) 
    #im1 = ax.scatter(growth_rate_list[0], growth_rate_list[1], c = growth_rate_list[2], s = 4, marker = 's', cmap = 'viridis_r')
    #im1 = ax[0].imshow(growth_rate_matrix.T, cmap = 'inferno', extent = (s0[0], s0[-1], s1[0], s1[-1]), origin = 'lower', interpolation = 'none')
    #im2 = ax[1].imshow(epidemic_size_matrix.T, cmap = 'copper', extent = (s0[0], s0[-1], s1[0], s1[-1]), origin = 'lower', interpolation = 'none')
    
    cp1 = ax[0].contour(s0, s1, growth_rate_matrix.T)
    ax[0].clabel(cp1, inline=True)

    cp2 = ax[1].contour(s0, s1, epidemic_size_matrix.T, levels = 20, linewidths = 0.5)
    ax[1].clabel(cp2, inline=True, fontsize = 6)
    #ax[0].plot(S0[thresh], S1[thresh], c = 'k', lw = 0, marker = '.', ls = '-.') 
    ax[1].plot(S0[thresh], S1[thresh], c = 'red', lw = 1, ls = '-.') 
    #ax[1].scatter(S0[thresh], S1[thresh], c = 'k', marker = ',', s = 0.25) 
        
    c_vis_list = [c_unconstrained, c_constrained]
    marker_shape = ['s', 'd']
    if type == 2:
        for j, c_vis in enumerate(c_vis_list):
            c_vis = np.array(c_vis)
            b_vis = np.array([b[k]*(1-c_vis[k])*(1-c_vis) for k in range(len(n))])  ## The new transmission matrix
            root_vis = so.root(FS, 0.9*n, args = (b_vis, n, gamma))
            s_vis = n - root_vis.x #The final sizes with intervention
            ax[0].scatter(s_vis[0], s_vis[1], label = r'$c = (%.2f$, $%.2f)$'%(c_vis[0], c_vis[1]), facecolor = 'none', edgecolor = 'purple', marker = marker_shape[j])
            print(c_vis, s_vis)
            print('b = ', b_vis)

    check_list = thresh#[0::5]
    temp = []
    for j, check in enumerate(check_list):
        c_check, c_status = c_typeII([n[0] - S0[check], n[1]-S1[check]], b, n)
        if c_status == True and np.all(c_check>-1e-3) and np.all(c_check<1):
            temp.append(check)
        #else:
        #    print('failed c = ', c_check)
    ax[1].plot(S0[temp], S1[temp], c = 'green', lw = 2, ls = '-') 
    #ax[0].plot(growth_rate_list[0][thresh], growth_rate_list[1][thresh], c = 'k', lw = 0.5, marker = ',', ls = '-.') 
    #ax[1].plot(growth_rate_list[0][thresh], growth_rate_list[1][thresh], c = 'k', lw = 1) 
        
    fig.colorbar(im1, ax=ax[0], location = 'bottom')
    fig.colorbar(im2, ax=ax[1], location = 'bottom')
    ax[0].legend(framealpha = 0.3, fancybox = False)
    ax[0].set_xlabel(r'$s_1^*$')
    ax[0].set_ylabel(r'$s_2^*$')
    ax[1].set_xlabel(r'$s_1^*$')
    ax[1].set_ylabel(r'$s_2^*$')
    ax[1].set_xlim(left = 0.01)
    ax[1].set_ylim(bottom = 0.01)
    fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 0.94811043 -5.15584358] [4.99999978e-01 1.00452204e-08]
b =  [[5.38505496e-03 1.11798427e+00]
 [7.98560192e-01 5.68416154e+01]]
[ 3.10008804e-01 -2.86213972e-13] [0.30958444 0.11610314]
b =  [[0.9521757  2.41496919]
 [1.72497799 1.5       ]]


C:\Users\Pratyush\AppData\Local\Temp/ipykernel_19064/3267254621.py:8: RuntimeWarning: divide by zero encountered in log
  return np.array([np.sum(b[k]*(1-c[k])*(1-c)*r) + np.log(1-r[k]/n[k]) for k in range(len(n))])


In [19]:
def SIR_equations(t, X, b, g, no_g): ##Define the dynamical equations for SIR model with 3 groups and arbitrary transmission
    S = X[0:no_g]
    I = X[no_g:2*no_g]
    R = X[2*no_g:3*no_g]

    dsdt = [(-S[k]*(np.sum(b[k]*I))) for k in range(no_g)]
    didt = [(S[k]*(np.sum(b[k]*I)) - g*I[k]) for k in range(no_g)]
    drdt = [g*I[k] for k in range(no_g)]
    return np.concatenate((dsdt, didt, drdt))

In [20]:
eps_i = 0.005 ## Threshold for i(t) at which intervention is imposed
eps_f = 0.5*1e-3 ## Threshold for i(t) below which intervention is released
ti = 0
tf = 80
dt = 0.05
T = np.arange(ti, tf, dt)
s_init, i_init, r_init = n - 1/3*1e-3, np.ones(no_g)*1/3*1e-3, np.zeros(no_g)
X_init = np.concatenate((s_init, i_init, r_init))
# 1/3*1e-3 is i(0) for each of the groups.

In [21]:
r_init

array([0., 0.])

In [22]:
X_init

array([6.99666667e-01, 2.99666667e-01, 3.33333333e-04, 3.33333333e-04,
       0.00000000e+00, 0.00000000e+00])

In [23]:
SOL_ord = solve_ivp(SIR_equations, [ti, tf], X_init, t_eval = T, args = (b, gamma, no_g)) # The ordinary trajectory (without intervention)
t_int_start_idx = np.where(np.sum(SOL_ord.y[no_g:2*no_g], axis = 0)-eps_i>=0)[0][0] ##Index value of time at which i(t) crossing eps_i
t_int_start = T[t_int_start_idx] ## Time at which i(t) crosses eps_i
X_init_int = SOL_ord.y[:, t_int_start_idx] ## The initial conditions for the intervention

In [24]:
## Now we implement the unconstrained solution intervention
SOL_int_unconstrained = solve_ivp(SIR_equations, [t_int_start, tf], X_init_int, t_eval = np.arange(t_int_start, tf, dt), args = (b_unconstrained, gamma, no_g)) 
t_int_end_uc_idx = np.where(np.sum(SOL_int_unconstrained.y[no_g:2*no_g], axis = 0) < eps_f)[0][0] ## Index value of time where i(t) reaching eps_f
t_int_end_uc = SOL_int_unconstrained.t[t_int_end_uc_idx]  ## Time at which i(t) crosses eps_f, and intervention is released
X_init_end_uc = SOL_int_unconstrained.y[:, t_int_end_uc_idx] ## Initial conditions for releasing the interventions
SOL_end_unconstrained = solve_ivp(SIR_equations, [t_int_end_uc, tf], X_init_end_uc, t_eval = np.arange(t_int_end_uc, tf-dt, dt), args = (b, gamma, no_g)) #Trajectory post intervention

## Now we implement the constrained solution intervention
SOL_int_constrained = solve_ivp(SIR_equations, [t_int_start, tf], X_init_int, t_eval = np.arange(t_int_start, tf, dt), args = (b_constrained, gamma, no_g)) 
t_int_end_c_idx = np.where(np.sum(SOL_int_constrained.y[no_g:2*no_g], axis = 0) < eps_f)[0][0] ## Index value of time where i(t) reaching eps_f
t_int_end_c = SOL_int_constrained.t[t_int_end_c_idx]  ## Time at which i(t) crosses eps_f, and intervention is released
X_init_end_c = SOL_int_constrained.y[:, t_int_end_c_idx] ##Initial conditions for releasing the interventions
SOL_end_constrained = solve_ivp(SIR_equations, [t_int_end_c, tf], X_init_end_c, t_eval = np.arange(t_int_end_c, tf-dt, dt), args = (b, gamma, no_g)) #Trajectory post intervention


In [25]:
t_int_end_c

25.45000000000002

In [26]:
fig, axs = plt.subplots(1, 2, figsize = (17*0.6, 5*0.6), dpi = 100)
#plt.suptitle(r'$\beta: %.2f$, $\gamma: %.2f$, $c: %.3f$'%(beta, gamma, c))

clrs = ['r', 'g', 'b']
for j, idx in enumerate(range(2*no_g, 3*no_g)):
    axs[0].plot(T, SOL_ord.y[idx].T, c = clrs[j], ls = '--', label = r'$r$, Group: %d'%j, lw = 0.75)
    axs[0].plot(SOL_int_unconstrained.t[0:t_int_end_uc_idx], SOL_int_unconstrained.y[idx, 0:t_int_end_uc_idx].T, c = clrs[j], ls = '-', lw = 0.75)
    axs[0].plot(SOL_end_unconstrained.t, SOL_end_unconstrained.y[idx].T, c = clrs[j], ls = '-', lw = 0.75)
    #axs[0].plot(SOL_int_unconstrained.t[0:t_int_end_uc_idx], SOL_int_unconstrained.y[idx, 0:t_int_end_uc_idx].T, c = clrs[j], ls = '-', lw = 0.75)
    #axs[0].plot(SOL_end_unconstrained.t, SOL_end_unconstrained.y[idx].T, c = clrs[j], ls = '-', lw = 0.75)

    axs[1].plot(T, SOL_ord.y[idx].T, c = clrs[j], ls = '--', label = r'$r$, Group: %d'%j, lw = 0.75)
    axs[1].plot(SOL_int_constrained.t[0:t_int_end_c_idx], SOL_int_constrained.y[idx, 0:t_int_end_c_idx].T, c = clrs[j], ls = '-', lw = 0.75)
    axs[1].plot(SOL_end_constrained.t, SOL_end_constrained.y[idx].T, c = clrs[j], ls = '-', lw = 0.75)
    #axs[1].plot(SOL_int_constrained.t[0:t_int_end_c_idx], SOL_int_constrained.y[idx, 0:t_int_end_c_idx].T, c = clrs[j], ls = '-', lw = 0.75)
    #axs[1].plot(SOL_end_constrained.t, SOL_end_constrained.y[idx].T, c = clrs[j], ls = '-', lw = 0.75)

# for j, idx in enumerate([0, 1, 2]):
#     axs[0].plot(T, SOL_ord.y[idx].T, c = clrs[j], ls = ':', label = r'$s$, Group: %d'%j, lw = 0.75)
#     axs[0].plot(SOL_int.t, SOL_int.y[idx].T, c = clrs[j], ls = '--', lw = 0.75)
#     axs[0].plot(SOL_rel.t, SOL_rel.y[idx].T, c = clrs[j], ls = '-', lw = 0.75)
# for j, idx in enumerate(range(no_g, 2*no_g)):
#     axs[1].plot(T, SOL_ord.y[idx].T, c = clrs[j], ls = ':', label = r'$i$, Group: %d'%j, lw = 0.9)
#     axs[1].plot(SOL_int_unconstrained.t[0:t_int_end_uc_idx], SOL_int_unconstrained.y[idx, 0:t_int_end_uc_idx].T, c = clrs[j], ls = '--', lw = 0.9)
#     axs[1].plot(SOL_end_unconstrained.t, SOL_end_unconstrained.y[idx].T, c = clrs[j], ls = '--', lw = 0.9)

axs[0].yaxis.tick_right()
axs[1].yaxis.tick_right()
axs[0].legend(loc = 'upper right')
axs[1].legend()
axs[0].set_xlim(0, 40)
axs[1].set_xlim(0, 40)
fig.tight_layout()


# print('Size of the groups: ', n)
# print('Original Transmission Matrix: \n', b)
# print('Final Sizes for the ordinary epidemic: ', FS_ord, '; Total: ', np.sum(FS_ord))
# print('Herd Immunity Point: s = ', s_HI)
# print('c = ', c)
# print('Transmission Matrix with intervention: \n', np.round(b_c, 3))
# print('Expected final size due to intervention: r = ', r_HI, '; Total: ', np.sum(r_HI))
# print('Observed final size due to intervention: r = ', SOL_rel.y[6:9, -1], '; Total: ', np.sum(SOL_rel.y[6:9, -1]))
# print('Details of the figure: dotted line is the original trajectory, dashed line is the intervention, orindary line is post intervention')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …